# Introduction to DataFrames
**[Bogumił Kamiński](http://bogumilkaminski.pl/about/), July 16, 2019**

In [1]:
using DataFrames # load package

## Working with CategoricalArrays

### Constructor

In [2]:
x = categorical(["A", "B", "B", "C"]) # unordered

4-element CategoricalArray{String,1,UInt32}:
 "A"
 "B"
 "B"
 "C"

In [3]:
y = categorical(["A", "B", "B", "C"], ordered=true) # ordered, by default order is sorting order

4-element CategoricalArray{String,1,UInt32}:
 "A"
 "B"
 "B"
 "C"

In [4]:
z = categorical(["A","B","B","C", missing]) # unordered with missings

5-element CategoricalArray{Union{Missing, String},1,UInt32}:
 "A"    
 "B"    
 "B"    
 "C"    
 missing

In [5]:
c = cut(1:10, 5) # ordered, into equal counts, possible to rename labels and give custom breaks

10-element CategoricalArray{String,1,UInt32}:
 "[1.0, 2.8)" 
 "[1.0, 2.8)" 
 "[2.8, 4.6)" 
 "[2.8, 4.6)" 
 "[4.6, 6.4)" 
 "[4.6, 6.4)" 
 "[6.4, 8.2)" 
 "[6.4, 8.2)" 
 "[8.2, 10.0]"
 "[8.2, 10.0]"

In [6]:
by(DataFrame(x=cut(randn(100000), 10)), :x, d -> DataFrame(n=nrow(d)), sort=true) # just to make sure it works right

,x,n
,Categorical…,Int64
1,"[-3.97022, -1.28501)",10000
2,"[-1.28501, -0.847175)",10000
3,"[-0.847175, -0.526623)",10000
4,"[-0.526623, -0.256371)",10000
5,"[-0.256371, -0.000949703)",10000
6,"[-0.000949703, 0.253573)",10000
7,"[0.253573, 0.52535)",10000
8,"[0.52535, 0.840664)",10000
9,"[0.840664, 1.27601)",10000


In [7]:
v = categorical([1,2,2,3,3]) # contains integers not strings

5-element CategoricalArray{Int64,1,UInt32}:
 1
 2
 2
 3
 3

In [8]:
Vector{Union{String, Missing}}(z) # sometimes you need to convert back to a standard vector

5-element Array{Union{Missing, String},1}:
 "A"    
 "B"    
 "B"    
 "C"    
 missing

### Managing levels

In [9]:
arr = [x,y,z,c,v]

5-element Array{CategoricalArray{T,1,UInt32,V,C,U} where U where C where V where T,1}:
 CategoricalString{UInt32}["A", "B", "B", "C"]                                                                                                                          
 CategoricalString{UInt32}["A", "B", "B", "C"]                                                                                                                          
 Union{Missing, CategoricalString{UInt32}}["A", "B", "B", "C", missing]                                                                                                 
 CategoricalString{UInt32}["[1.0, 2.8)", "[1.0, 2.8)", "[2.8, 4.6)", "[2.8, 4.6)", "[4.6, 6.4)", "[4.6, 6.4)", "[6.4, 8.2)", "[6.4, 8.2)", "[8.2, 10.0]", "[8.2, 10.0]"]
 CategoricalValue{Int64,UInt32}[1, 2, 2, 3, 3]                                                                                                                          

In [10]:
isordered.(arr) # chcek if categorical array is orderd

5-element BitArray{1}:
 false
  true
 false
  true
 false

In [11]:
ordered!(x, true), isordered(x) # make x ordered

(CategoricalString{UInt32}["A", "B", "B", "C"], true)

In [12]:
ordered!(x, false), isordered(x) # and unordered again

(CategoricalString{UInt32}["A", "B", "B", "C"], false)

In [13]:
levels.(arr) # list levels

5-element Array{Array{T,1} where T,1}:
 ["A", "B", "C"]                                                        
 ["A", "B", "C"]                                                        
 ["A", "B", "C"]                                                        
 ["[1.0, 2.8)", "[2.8, 4.6)", "[4.6, 6.4)", "[6.4, 8.2)", "[8.2, 10.0]"]
 [1, 2, 3]                                                              

In [14]:
unique.(arr) # missing will be included

5-element Array{Array{T,1} where T,1}:
 ["A", "B", "C"]                                                        
 ["A", "B", "C"]                                                        
 Union{Missing, String}["A", "B", "C", missing]                         
 ["[1.0, 2.8)", "[2.8, 4.6)", "[4.6, 6.4)", "[6.4, 8.2)", "[8.2, 10.0]"]
 [1, 2, 3]                                                              

In [15]:
y[1] < y[2] # can compare as y is ordered

true

In [16]:
v[1] < v[2] # not comparable, v is unordered although it contains integers

ArgumentError: ArgumentError: Unordered CategoricalValue objects cannot be tested for order using <. Use isless instead, or call the ordered! function on the parent array to change this

In [17]:
y[2] < "A" # comparison against type underlying categorical value is possible

false

In [18]:
y[2] < "Z" # but it is treated as a level, and thus only valid levels are allowed

KeyError: KeyError: key "Z" not found

In [19]:
levels!(y, ["C", "B", "A"]) # you can reorder levels, mostly useful for ordered CategoricalArrays

4-element CategoricalArray{String,1,UInt32}:
 "A"
 "B"
 "B"
 "C"

In [20]:
y[1] < y[2] # observe that the order is changed

false

In [21]:
y[1] < "B" # level ordering is respected also when comparing with an underlying type

false

In [22]:
levels!(z, ["A", "B"]) # you have to specify all levels that are present

ArgumentError: ArgumentError: cannot remove level "C" as it is used at position 4 and allow_missing=false.

In [23]:
levels!(z, ["A", "B"], allow_missing=true) # unless the underlying array allows for missings and force removal of levels

5-element CategoricalArray{Union{Missing, String},1,UInt32}:
 "A"    
 "B"    
 "B"    
 missing
 missing

In [24]:
z[1] = "B"
z # now z has only "B" entries

5-element CategoricalArray{Union{Missing, String},1,UInt32}:
 "B"    
 "B"    
 "B"    
 missing
 missing

In [25]:
levels(z) # but it remembers the levels it had (the reason is mostly performance)

2-element Array{String,1}:
 "A"
 "B"

In [26]:
droplevels!(z) # this way we can clean it up
levels(z)

1-element Array{String,1}:
 "B"

### Data manipulation

In [27]:
x, levels(x)

(CategoricalString{UInt32}["A", "B", "B", "C"], ["A", "B", "C"])

In [28]:
x[2] = "0"
x, levels(x) # new level added at the end (works only for unordered)

(CategoricalString{UInt32}["A", "0", "B", "C"], ["A", "B", "C", "0"])

In [29]:
v, levels(v)

(CategoricalValue{Int64,UInt32}[1, 2, 2, 3, 3], [1, 2, 3])

In [30]:
v[1] + v[2] # even though underlying data is Int, we cannot operate on it

MethodError: MethodError: no method matching +(::CategoricalValue{Int64,UInt32}, ::CategoricalValue{Int64,UInt32})
Closest candidates are:
  +(::Any, ::Any, !Matched::Any, !Matched::Any...) at operators.jl:502

In [31]:
Vector{Int}(v) # you have either to retrieve the data by conversion (may be expensive)

5-element Array{Int64,1}:
 1
 2
 2
 3
 3

In [32]:
get(v[1]) + get(v[2]) # or get a single value

3

In [33]:
get.(v) # this will work for arrays witout missings

5-element Array{Int64,1}:
 1
 2
 2
 3
 3

In [34]:
get.(z) # but will fail on missing values

MethodError: MethodError: no method matching get(::Missing)
Closest candidates are:
  get(!Matched::Base.EnvDict, !Matched::AbstractString, !Matched::Any) at env.jl:77
  get(!Matched::Base.TTY, !Matched::Symbol, !Matched::Any) at stream.jl:482
  get(!Matched::REPL.Terminals.TTYTerminal, !Matched::Any, !Matched::Any) at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.1\REPL\src\Terminals.jl:176
  ...

In [35]:
Vector{Union{String, Missing}}(z) # you have to do the conversion

5-element Array{Union{Missing, String},1}:
 "B"    
 "B"    
 "B"    
 missing
 missing

In [36]:
z[1]*z[2], z.^2 # the only exception are CategoricalArrays based on String - you can operate on them as on normal strings

("BB", Union{Missing, String}["BB", "BB", "BB", missing, missing])

In [37]:
recode([1,2,3,4,5,missing], 1=>10) # recode some values in an array; has also in place recode! equivalent

6-element Array{Union{Missing, Int64},1}:
 10       
  2       
  3       
  4       
  5       
   missing

In [38]:
recode([1,2,3,4,5,missing], "a", 1=>10, 2=>20) # here we provided a default value for not mapped recodings

6-element Array{Any,1}:
 10       
 20       
   "a"    
   "a"    
   "a"    
   missing

In [39]:
recode([1,2,3,4,5,missing], 1=>10, missing=>"missing") # to recode Missing you have to do it explicitly

6-element Array{Any,1}:
 10         
  2         
  3         
  4         
  5         
   "missing"

In [40]:
t = categorical([1:5; missing])
t, levels(t)

(Union{Missing, CategoricalValue{Int64,UInt32}}[1, 2, 3, 4, 5, missing], [1, 2, 3, 4, 5])

In [41]:
recode!(t, [1,3]=>2)
t, levels(t) # note that the levels are dropped after recode

(Union{Missing, CategoricalValue{Int64,UInt32}}[2, 2, 2, 4, 5, missing], [2, 4, 5])

In [42]:
t = categorical([1,2,3], ordered=true)
levels(recode(t, 2=>0, 1=>-1)) # and if you introduce a new levels they are added at the end in the order of appearance

3-element Array{Int64,1}:
  3
  0
 -1

In [43]:
t = categorical([1,2,3,4,5], ordered=true) # when using default it becomes the last level
levels(recode(t, 300, [1,2]=>100, 3=>200))

3-element Array{Int64,1}:
 100
 200
 300

### Comparisons

In [44]:
x = categorical([1,2,3])
xs = [x, categorical(x), categorical(x, ordered=true), categorical(x, ordered=true)]
levels!(xs[2], [3,2,1])
levels!(xs[4], [2,3,1])
[a == b for a in xs, b in xs] # all are equal - comparison only by contents

4×4 Array{Bool,2}:
 true  true  true  true
 true  true  true  true
 true  true  true  true
 true  true  true  true

In [45]:
signature(x::CategoricalArray) = (x, levels(x), isordered(x)) # this is actually the full signature of CategoricalArray
# all are different, notice that x[1] and x[2] are unordered but have a different order of levels
[signature(a) == signature(b) for a in xs, b in xs]

4×4 Array{Bool,2}:
  true  false  false  false
 false   true  false  false
 false  false   true  false
 false  false  false   true

In [46]:
x[1] < x[2] # you cannot compare elements of unordered CategoricalArray

ArgumentError: ArgumentError: Unordered CategoricalValue objects cannot be tested for order using <. Use isless instead, or call the ordered! function on the parent array to change this

In [47]:
t[1] < t[2] # but you can do it for an ordered one

true

In [48]:
isless(x[1], x[2]) # isless works within the same CategoricalArray even if it is not ordered

true

In [49]:
y = deepcopy(x) # but not across categorical arrays
isless(x[1], y[2])

ArgumentError: ArgumentError: CategoricalValue objects with different pools cannot be tested for order

In [50]:
isless(get(x[1]), get(y[2])) # you can use get to make a comparison of the contents of CategoricalArray

true

In [51]:
x[1] == y[2] # equality tests works OK across CategoricalArrays

false

### Categorical columns in a DataFrame

In [52]:
df = DataFrame(x = 1:3, y = 'a':'c', z = ["a","b","c"])

,x,y,z
,Int64,Char,String
1,1,'a',a
2,2,'b',b
3,3,'c',c


In [53]:
categorical!(df) # converts all eltype(AbstractString) columns to categorical

,x,y,z
,Int64,Char,Categorical…
1,1,'a',a
2,2,'b',b
3,3,'c',c


In [54]:
describe(df)

,variable,mean,min,median,max,nunique,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Union…,Nothing,DataType
1,x,2.0,1,2.0,3,,,Int64
2,y,,'a',,'c',3,,Char
3,z,,a,,c,3,,CategoricalString{UInt32}


In [55]:
categorical!(df, :x) # manually convert to categorical column :x

,x,y,z
,Categorical…,Char,Categorical…
1,1,'a',a
2,2,'b',b
3,3,'c',c


In [56]:
describe(df)

,variable,mean,min,median,max,nunique,nmissing,eltype
,Symbol,Nothing,Any,Nothing,Any,Int64,Nothing,DataType
1,x,,1,,3,3,,"CategoricalValue{Int64,UInt32}"
2,y,,'a',,'c',3,,Char
3,z,,a,,c,3,,CategoricalString{UInt32}
